
EMOTİON DETECTİON SEMESTER PROJECT

In [35]:
import pandas as pd 

In [28]:
import cv2

In [38]:
import numpy as np 
from sklearn.model_selection import train_test_split

In [4]:
FILE_PATH = 'fer2013.csv'

In [5]:
image_size = (48, 48)

In [6]:
data = pd.read_csv(FILE_PATH)    

In [7]:
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [8]:
pixels = data['pixels'].tolist()

In [9]:
width, height = image_size

In [10]:
# görüntüleri ve duyguları yükler 
faces = []

for p in pixels:
    face = [int(pix) for pix in p.split(' ')]
    face = np.asarray(face).reshape(width, height)
    face = cv2.resize(face.astype('uint8'), image_size)
    faces.append(face.astype('float32'))

In [11]:
len(faces)

35887

In [12]:
faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)

In [13]:
emotions = pd.get_dummies(data['emotion']).values
emotions.shape


(35887, 7)

In [14]:
# normalizasyonu gerçekleştirir.0 ile 1 arasında değer almasını sağlar.2 ile çarpıp 0.5 ile çıkartarak da -1 ile 1 arasında değere dönüştürür. 
def preprocess(x, v2=True):  
    x = x.astype('float32')
    x = x/255.0
    if v2:
        x = (x - 0.5)*2.0
    return x

In [15]:
faces = preprocess(faces)

In [16]:
#print('örnek eğitilmiş resimleri göster')
#for image in np.arange(0,10):
#    cv2.namedWindow('örnek eğitilmiş resimleri göster', cv2.WINDOW_NORMAL)
#    cv2.imshow('örnek eğitilmiş resimleri göster',faces[image])
#    cv2.waitKey(500)
#    cv2.destroyAllWindows()

In [17]:
# Eğitme ve test etme için 4 değer döndürür.
xtrain, xtest,ytrain,ytest = train_test_split(faces, emotions,test_size=0.2,shuffle=True)


# CNN model : Mini Xception

In [18]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519

In [19]:
from keras.preprocessing.image import ImageDataGenerator

In [20]:
from keras import layers
from keras.layers import Activation, Convolution2D, Conv2D, Dropout, AveragePooling2D, BatchNormalization, GlobalAveragePooling2D, Flatten, Input, MaxPooling2D, SeparableConv2D

In [21]:
from keras.models import Model

In [22]:
from keras.regularizers import l2

In [25]:
#parameters 

batch_size = 32
epochs = 100
image_shape = (48, 48, 1)
verbose = True 
num_class = 7
patience = 50  # sonrasında eğitimin durdurulacağı gelişme olmayan dönemlerin sayısı
base_path = 'models/'
l2_regularization = 0.01


In [24]:
data_generator = ImageDataGenerator(featurewise_center=False, featurewise_std_normalization=False, rotation_range=10, 
                                    width_shift_range=0.1, height_shift_range=0.1, zoom_range=.1, horizontal_flip=True)

In [25]:
regularization = l2(l2_regularization)

In [26]:
# model
image_input = Input(image_shape)
x = Conv2D(filters=8, kernel_size=(3,3), strides=(1,1), kernel_regularizer=regularization, use_bias=False)(image_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(filters=8, kernel_size=(3,3), strides=(1,1), kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# module 1
# residual module 
residual = Conv2D(filters=16, kernel_size=(1,1), strides=(2,2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(filters=16, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(filters=16, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
x = layers.add([x,residual])

# module 2
# residual module 
residual = Conv2D(filters=32, kernel_size=(1,1), strides=(2,2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(filters=32, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(filters=32, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
x = layers.add([x,residual])

# module 3
# residual module 
residual = Conv2D(filters=64, kernel_size=(1,1), strides=(2,2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(filters=64, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(filters=64, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
x = layers.add([x,residual])

# module 4
# residual module 
residual = Conv2D(filters=128, kernel_size=(1,1), strides=(2,2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(filters=128, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(filters=128, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
x = layers.add([x,residual])

x = Conv2D(filters=num_class, kernel_size=(3,3), padding='same')(x)
x = GlobalAveragePooling2D()(x)

output = Activation('softmax', name='predictions')(x)



# Train

In [27]:
# loss en son hatalarımızı hesaplamamızı sağlar.(Geriye doğru türev alır) Az ise iyi eğitilmiş olur.
# Optimizer ile parametremizi buluruz.
# Metrics modelimizin sonucunu değerlendirmeyi sağlar

model = Model(image_input, output)
model.compile(loss = "categorical_crossentropy", optimizer = "Adam", metrics = ["acc"])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 46, 46, 8)    72          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 46, 46, 8)    32          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 46, 46, 8)    0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [28]:
# callbacks 
log_file_path = base_path + '_emotion_training.log'
csv_logger = CSVLogger(log_file_path, append=False)

early_stop = EarlyStopping(monitor='val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=int(patience/4), verbose=verbose)

trained_models_path = base_path + '_mini_xception'
model_names = trained_models_path + '.{epoch:02d}_{val_acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(filepath=model_names, monitor='val_loss', verbose=verbose, save_best_only=True)

callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

In [29]:
# Eğitmenin gerçekleşmesi sağlanır. (epochs kaç kere eğiteleceğini gösterir,batch resimlerin kaç grup halde train edileceğini gösterir.)

# model.fit(xtrain, ytrain, epochs=35, batch_size=64)

model.fit_generator(data_generator.flow(xtrain, ytrain,batch_size),
                        steps_per_epoch=len(xtrain) / batch_size,
                        epochs=epochs, verbose=1, callbacks=callbacks,
                        validation_data=(xtest,ytest))

# Değerlendirme işlemleri gerçekleştirilir.
score_train = model.evaluate(xtrain, ytrain)
print("Eğitim doğruluğu: %",score_train[1]*100) # score[0] kaybı verir.
    
score_test = model.evaluate(xtest, ytest)
print("Test doğruluğu: %",score_test[1]*100) 


Epoch 1/100
898/897 [==============================] - 337s 375ms/step - loss: 1.7716 - acc: 0.3331 - val_loss: 1.8491 - val_acc: 0.3402

Epoch 00001: val_loss improved from inf to 1.84906, saving model to models/_mini_xception.01_0.34.hdf5
Epoch 2/100
898/897 [==============================] - 326s 363ms/step - loss: 1.4861 - acc: 0.4453 - val_loss: 1.4928 - val_acc: 0.4462

Epoch 00002: val_loss improved from 1.84906 to 1.49281, saving model to models/_mini_xception.02_0.45.hdf5
Epoch 3/100
898/897 [==============================] - 313s 349ms/step - loss: 1.3772 - acc: 0.4853 - val_loss: 1.3385 - val_acc: 0.5024

Epoch 00003: val_loss improved from 1.49281 to 1.33848, saving model to models/_mini_xception.03_0.50.hdf5
Epoch 4/100
898/897 [==============================] - 320s 356ms/step - loss: 1.3087 - acc: 0.5100 - val_loss: 1.3388 - val_acc: 0.5057

Epoch 00004: val_loss did not improve from 1.33848
Epoch 5/100
898/897 [==============================] - 326s 363ms/step - loss: 

# Bir görüntüdeki bir yüzün duygularını algılama

In [62]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import imutils
import sys 

In [63]:
detection_model_path = 'haarcascade_frontalface_default.xml'
emotion_recognition_model_path = base_path + '_mini_xception.100_0.65.hdf5'
image_path = 'duygu-1.jpg'

In [64]:
face_detection = cv2.CascadeClassifier(detection_model_path)

In [65]:
emotion_classifier = load_model(emotion_recognition_model_path)

In [66]:
emotions = ['angry', 'disgust', 'scared', 'happy', 'sad', 'surprised', 'neutral']

In [67]:
color_frame = cv2.imread(image_path)
gray_frame = cv2.imread(image_path, 0)

In [68]:
cv2.imshow('Input test image', color_frame)
cv2.waitKey(1000)
cv2.destroyAllWindows()

In [71]:
detected_faces = face_detection.detectMultiScale(color_frame, scaleFactor=1.1, minNeighbors=5, 
                                        minSize=(30,30), flags=cv2.CASCADE_SCALE_IMAGE)
print('Number of faces detected : ', len(detected_faces))

if len(detected_faces)>0:

 # Birden fazla yüz olduğu durumda   
    detected_faces = sorted(detected_faces, reverse=True, key=lambda x: (x[2]-x[0])*(x[3]-x[1]))[3] 
    (fx, fy, fw, fh) = detected_faces
    
    im = gray_frame[fy:fy+fh, fx:fx+fw]
    im = cv2.resize(im, (48,48))   
    im = im.astype("float")/255.0
    im = img_to_array(im)
    im = np.expand_dims(im, axis=0)
    
    preds = emotion_classifier.predict(im)[0]
    emotion_probability = np.max(preds)
    label = emotions[preds.argmax()]
    
    cv2.putText(color_frame, label, (fx, fy-10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
    cv2.rectangle(color_frame, (fx, fy), (fx + fw, fy + fh),(0, 0, 255), 2)

cv2.imshow('Input test image', color_frame)
cv2.imwrite('output_'+image_path.split('/')[-1], color_frame)
cv2.waitKey(1000)
cv2.destroyAllWindows()

Number of faces detected :  18


# Bir videodaki yüzlerin duygularını algılama

In [38]:
cv2.namedWindow('emotion_recognition')
camera = cv2.VideoCapture(0)  ## bilgisayar kamerası kullanılır. 
## camera = cv2.VideoCapture('various_emotions.mp4')  # video dosyasından okur.

sz = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))

fourcc = cv2.VideoWriter_fourcc(*'mpeg')

out = cv2.VideoWriter()
out.open('output_various_emotions.mp4',fourcc, 15, sz, True) # videoya yazmaya başlar


# while True: # kameradan video okunurken while kullanılır
while(camera.read()[0]):  # dosyadan okurken de while kullanılır.
    color_frame = camera.read()[1]
    color_frame = imutils.resize(color_frame,width=min(720, color_frame.shape[1]))
    
    
    gray_frame = cv2.cvtColor(color_frame, cv2.COLOR_BGR2GRAY)
    detected_faces = face_detection.detectMultiScale(gray_frame,scaleFactor=1.1,minNeighbors=5,minSize=(30,30),flags=cv2.CASCADE_SCALE_IMAGE)
    
    
    canvas = np.zeros((250, 300, 3), dtype="uint8")
    frameClone = color_frame.copy()    

    
    if len(detected_faces)>0:

        detected_faces = sorted(detected_faces, reverse=True, key=lambda x: (x[2]-x[0])*(x[3]-x[1]))[0] # birden fazla yüz varsa
        (fx, fy, fw, fh) = detected_faces

        im = gray_frame[fy:fy+fh, fx:fx+fw]
        im = cv2.resize(im, (48,48))  # model 48*48 piksel görüntü üzerinde eğitilmiştir. 
        im = im.astype("float")/255.0
        im = img_to_array(im)
        im = np.expand_dims(im, axis=0)

        preds = emotion_classifier.predict(im)[0]
        emotion_probability = np.max(preds)
        label = emotions[preds.argmax()]

        cv2.putText(color_frame, label, (fx, fy-10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        cv2.rectangle(color_frame, (fx, fy), (fx + fw, fy + fh),(0, 0, 255), 2)

    
    for (i, (emotion, prob)) in enumerate(zip(emotions, preds)):
        # construct the label text
        text = "{}: {:.2f}%".format(emotion, prob * 100)
        w = int(prob * 300)
        
        cv2.rectangle(canvas, (7, (i * 35) + 5), (w, (i * 35) + 35), (0, 50, 100), -1)
        cv2.putText(canvas, text, (10, (i * 35) + 23), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 1)
        cv2.putText(frameClone, label, (fx, fy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (100, 150, 100), 2)
        cv2.rectangle(frameClone, (fx, fy), (fx + fw, fy + fh), (100, 100, 100), 2)
    
    out.write(frameClone)
    out.write(canvas)
    
    cv2.imshow('emotion_recognition', frameClone)
    cv2.imshow("Probabilities", canvas)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
camera.release()
out.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 